<a href="https://colab.research.google.com/github/fernandofsilva/LSTM_Option_Pricing/blob/main/notebooks/00_pre_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#@title Carregar as bibliotecas base
import pandas as pd
import zipfile
import os

Os dados das cotações de todos os ativos negociadas na [B3](http://www.b3.com.br), estão disponíveis no [link](http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/) e precisam ser parseados de acordo com as seguintes instruções [link](http://www.b3.com.br/en_us/market-data-and-indices/data-services/market-data/historical-data/equities/historical-quote-data/).

Para essa tarefa é necessário utilizado o pacote [bovespa](https://github.com/fernandofsilva/bovespa), ele é a versão atualizada por mim do pacote desenvolvido a 5 anos atrás [link](https://github.com/thypad/bovespa).

In [ ]:
#@title Instalar pacote Bovespa
!pip install git+https://github.com/fernandofsilva/bovespa

  Cloning https://github.com/fernandofsilva/bovespa to /tmp/pip-req-build-e9fgd978
  Running command git clone -q https://github.com/fernandofsilva/bovespa /tmp/pip-req-build-e9fgd978
  Created wheel for bovespa: filename=bovespa-0.1.0-cp36-none-any.whl size=9211 sha256=dcf90420571dcd830a7478c6e42d2ddd780ba1f77ed193e18e07cbf5c9881cb4
  Stored in directory: /tmp/pip-ephem-wheel-cache-plmfke1w/wheels/cb/3c/9b/27644c70e14f3c2a2dd611dcfcf8f4f6856e4577f3f881eff4
Successfully built bovespa


Os dados estão salvos dentro do meu Google Drive, porém estão disponíveis através do site da bovespa e também dentro do [Github](https://github.com/fernandofsilva/LSTM_Option_Pricing/tree/main/data) do projeto.

In [ ]:
#@title Descompactor os arquivos

# Caminho para os arquivos
path = '/content/drive/My Drive/Mestrado/data/'

# Listar os arquivos dentro do diretório
files = os.listdir(path)

# Filtrar para pegar somente os dados de contação
files = list(filter(lambda file: 'COTAHIST' in file, files))

# Loop sobe os arquivos, decompactando e salvado local
for file in files:
    with zipfile.ZipFile(path + file, 'r') as zip_ref:
        zip_ref.extractall('')

In [ ]:
#@title Amostra dos dados
!head -10 COTAHIST_A2020.TXT

00COTAHIST.2020BOVESPA 20201113                                                                                                                                                                                                                      
012020010202AALR3       010ALLIAR      ON      NM   R$  000000000182900000000019000000000001828000000000186800000000019000000000001899000000000190102443000000000000585800000000001094619600000000000000009999123100000010000000000000BRAALRACNOR6101
012020010202AAPL34      010APPLE       DRN          R$  000000001200000000000121340000000012000000000001207300000000121340000000010550000000001213400009000000000000012700000000000153338000000000000000009999123100000010000000000000BRAAPLBDR004131
012020010202ABCB4       010ABC BRASIL  PN  EJ  N2   R$  000000000200000000000020300000000001982000000000200500000000020300000000002008000000000203003979000000000000870400000000001745787800000000000000009999123100000010000000000000BRABCBACNPR4133
012020010202ABEV

# Parser dos dados

Parsear os dados e salvar em format .Csv

In [ ]:
#@title Parser
import bovespa

# Corrigir os nomes dos arquivos
files = list(map(lambda file: file.replace('.zip', '.TXT'), files))

# Loop sobre todos os arquivos
for file in files:

    # Parse dos arquivos
    bf = bovespa.stock_history([file])

    # Armazenar os arquivos em Csv
    bf.to_csv(f'{file[:-4]}.csv')

OrderedDict([('TIPREG', '00'), ('NOMARQ', 'COTAHIST.2019'), ('CODORI', 'BOVESPA'), ('DATGER', datetime.date(2019, 12, 30)), ('RESERV', '')])
OrderedDict([('TIPREG', '00'), ('NOMARQ', 'COTAHIST.2020'), ('CODORI', 'BOVESPA'), ('DATGER', datetime.date(2020, 11, 13)), ('RESERV', '')])
OrderedDict([('TIPREG', '00'), ('NOMARQ', 'COTAHIST.2018'), ('CODORI', 'BOVESPA'), ('DATGER', datetime.date(2018, 12, 28)), ('RESERV', '')])
OrderedDict([('TIPREG', '00'), ('NOMARQ', 'COTAHIST.2017'), ('CODORI', 'BOVESPA'), ('DATGER', datetime.date(2017, 12, 29)), ('RESERV', '')])
OrderedDict([('TIPREG', '00'), ('NOMARQ', 'COTAHIST.2016'), ('CODORI', 'BOVESPA'), ('DATGER', datetime.date(2016, 12, 29)), ('RESERV', '')])
OrderedDict([('TIPREG', '00'), ('NOMARQ', 'COTAHIST.2015'), ('CODORI', 'BOVESPA'), ('DATGER', datetime.date(2015, 12, 30)), ('RESERV', '')])


# Selecionar colunas e salvar Csv

In [ ]:
#@title Ler os arquivos
# Corrigir o nome dos arquivos
files = list(map(lambda file: file.replace('.TXT', '.csv'), files))

# Nome das colunas
cols = ['DATPRG', 'CODNEG', 'PREULT', 'PREEXE', 'DATVEN', 'TPMERC']

# Iniciar uma lista
dataframe_list = []

# Loop sobre os arquivos
for file in files:

    # Adicionar na lista cada dataframe
    dataframe_list.append(pd.read_csv(file, usecols=cols))

# Concatenar todos os arquivos em formato de coluna
data = pd.concat(dataframe_list, axis=0)

In [ ]:
#@title Formatar os dados
# Converter para tipo de data e tempo
data.loc[data['DATVEN'] == '9999-12-31', 'DATVEN'] = data.loc[data['DATVEN'] == '9999-12-31', 'DATPRG']
data[['DATPRG', 'DATVEN']] = data[['DATPRG', 'DATVEN']].apply(pd.to_datetime)

# Dicionário para renomer as colunas
dict_ref = {
    'DATPRG': 'data_pregao',
    'CODNEG': 'codigo',
    'PREULT': 'preco',
    'PREEXE': 'preco_exercicio',
    'DATVEN': 'data_vencimento',
    'TPMERC': 'mercado'
}

# Renomear oas colunas
data = data.rename(columns=dict_ref)

# Dicionário com os códigos dos papéis
market = {
    10: 'VISTA',
    12: 'EXERCÍCIO DE OPÇÕES DE COMPRA',
    13: 'EXERCÍCIO DE OPÇÕES DE VENDA',
    17: 'LEILÃO',
    20: 'FRACIONÁRIO',
    30: 'TERMO',
    50: 'FUTURO COM RETENÇÃO DE GANHO',
    60: 'FUTURO COM MOVIMENTAÇÃO CONTÍNUA',
    70: 'OPÇÕES DE COMPRA',
    80: 'OPÇÕES DE VENDA',
}

# Renomear códigos nas colunas
data['mercado'] = data['mercado'].replace(market)

data.head()

,data_pregao,codigo,mercado,preco,preco_exercicio,data_vencimento
0,2019-01-02,AALR3,VISTA,13.25,0.0,2019-01-02
1,2019-01-02,AAPL34,VISTA,60.41,0.0,2019-01-02
2,2019-01-02,ABBV34,VISTA,342.84,0.0,2019-01-02
3,2019-01-02,ABCB4,VISTA,17.12,0.0,2019-01-02
4,2019-01-02,ABEV3,VISTA,16.15,0.0,2019-01-02


O arquivo final é salvo no meu google drive, porém uma cópia está disponível no [link](https://github.com/fernandofsilva/LSTM_Option_Pricing/tree/main/data)

In [ ]:
#@title Salvar Csv
data.to_csv(path + 'cotacao_historica.csv.gz', index=False, compression='gzip')

# CDI

Os dados da taxa de juros estão disponíveis no [ftp](ftp://ftp.cetip.com.br/MediaCDI) da B3.

In [ ]:
#@title Download dos dados no FTP B3
from ftplib import FTP

# Função para acrescentar os dados na lista
dados = []
def handle_binary(mais_dados):
    dados.append(mais_dados)

# Endereço dos dados
host = "ftp.cetip.com.br"

# Pegar a lista de arquivos disponíveis dentro do FTP
with FTP(host) as ftp:
    ftp.login()
    files = ftp.nlst('MediaCDI')
    for file in files:
        resp = ftp.retrbinary("RETR " + file, callback=handle_binary)

In [ ]:
#@title Formatando os dados
import re

# Formatar dados
def f(item):
    item, *_ = item.decode('utf-8').split(' ')
    return float(item[:-2] + '.' + item[-2:])

taxa = map(f, dados)

def s(item):
    return re.search(r"(\d+)", item).group(0)

data = map(s, files)

In [ ]:
#@title Criar Csv
import pandas as pd

cdi = pd.DataFrame({'data': data, 'taxa_juros': taxa})
cdi['data'] = pd.to_datetime(cdi['data'], format='%Y%m%d')
cdi = cdi.sort_values(by='data')
cdi.head()

,data,taxa_juros
759,2012-08-20,7.85
771,2012-08-21,7.82
819,2012-08-22,7.81
1000,2012-08-23,7.81
1650,2012-08-24,7.82


O arquivo final é salvo no meu google drive, porém uma cópia está disponível no [link](https://github.com/fernandofsilva/LSTM_Option_Pricing/tree/main/data)

In [ ]:
#@title Salvar Csv
cdi.to_csv(path + 'cdi.csv.gz', index=False, compression='gzip')